# Silver Layer - Transformação de Clientes

In [0]:
 %run ./00_Setup_Environment

In [0]:
%sql
--SELECT * FROM bronze.customer

In [0]:
%sql
--SELECT NameStyle, count(*) FROM bronze.customer GROUP BY NameStyle

In [0]:
df_customer = spark.table("bronze.customer")
df_customer_address = spark.table("bronze.customer_address")  # Já transformado

In [0]:
#df_customer_addresses.display()

In [0]:
#df_customer.display()

In [0]:
df_silver_customer = df_customer.alias("c") \
    .select(
        # Identificadores
        col("c.CustomerID").alias("customer_id"),

        # Dados pessoais
        col("NameStyle").alias("name_style"),
        initcap(trim(col("c.Title"))).alias("title"),
        initcap(trim(col("c.FirstName"))).alias("first_name"),
        initcap(trim(col("c.MiddleName"))).alias("middle_name"),
        initcap(trim(col("c.LastName"))).alias("last_name"),
        col("c.Suffix").alias("suffix"),
        concat_ws(" ",
                  when(initcap(trim(col("c.Title"))).isNotNull(), col("c.Title")).otherwise(""),
                  col("c.FirstName"),
                  when(initcap(trim(col("c.MiddleName"))).isNotNull(), col("c.MiddleName")).otherwise(""),
                  col("c.LastName"),
                  when(initcap(trim(col("c.Suffix"))).isNotNull(), col("c.Suffix")).otherwise("")).alias("full_name"),
        col("c.EmailAddress").alias("email_address"),
        col("c.Phone").alias("phone"),

        #sobre local de compra
        col("c.CompanyName").alias("company_name"),
        col("c.SalesPerson").alias("sales_user_name"),
        col("c.rowguid").alias("customer_rowguid"),
       
        # Metadados
        col("c.ModifiedDate").cast("timestamp").alias("source_modified_date"),
        current_timestamp().alias("processed_timestamp"),
        
        #validação
        when((col("c.FirstName").isNotNull())  & (col("c.LastName").isNotNull()), True).otherwise(False).alias("is_valid_customer")
    ) \
    .distinct()

In [0]:
df_silver_customer.display()

In [0]:
path = f"{silver_path}/customer"
df_silver_customer.write \
    .mode("overwrite") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .save(path)

spark.sql("""USE adventureworks.silver""")
df_silver_customer.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("customer")

count = df_silver_customer.count()
log_etl("customer", "silver", "SUCCESS", count)

print(f" Silver Customer: {count} registros")